In [3]:
#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
pandas version: 1.3.4
matplotlib version: 3.4.3
NumPy version: 1.20.3
SciPy version: 1.7.1
IPython version: 7.29.0
scikit-learn version: 0.24.2
-------------------------


In [4]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [5]:
data_raw = pd.read_csv('train.csv')

data_val  = pd.read_csv('test.csv')


#to play with our data we'll create a copy
#remember python assignment or equal passes by reference vs values, so we use the copy function: https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
data1 = data_raw.copy(deep = True)

#however passing by reference is convenient, because we can clean both datasets at once
data_cleaner = [data1, data_val]

#preview data
print (data_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   listing_id      26048 non-null  int64  
 1   name            26048 non-null  object 
 2   street          26048 non-null  object 
 3   type            26048 non-null  object 
 4   model           24768 non-null  object 
 5   market_segment  26048 non-null  object 
 6   type_of_area    26048 non-null  object 
 7   bedrooms        25644 non-null  object 
 8   bathrooms       24812 non-null  float64
 9   district        26048 non-null  int64  
 10  region          26048 non-null  object 
 11  planning_area   26048 non-null  object 
 12  subszone        26048 non-null  object 
 13  lat             26048 non-null  float64
 14  lng             26048 non-null  float64
 15  tenure          25761 non-null  object 
 16  built_year      16005 non-null  float64
 17  no_of_units     25006 non-null 

In [98]:
print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')

Train columns with null values:
 listing_id            0
name                  0
street                0
type                  0
model              1280
market_segment        0
type_of_area          0
bedrooms            404
bathrooms          1236
district              0
region                0
planning_area         0
subszone              0
lat                   0
lng                   0
tenure              287
built_year        10043
no_of_units        1042
area_size             2
eco_category          0
accessibility         0
date_listed           0
price                 0
dtype: int64
----------
Test/Validation columns with null values:
 listing_id           0
name                 0
street               0
type                 0
model              388
market_segment       0
type_of_area         0
bedrooms           118
bathrooms          337
district             0
region               0
planning_area        0
subszone             0
lat                  0
lng                  0
ten

,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price
count,2.604800e+04,26048,26048,26048,24768,26048,26048,25644,24812.000000,26048.000000,...,26048.000000,26048.000000,25761,16005.000000,25006.000000,26046.000000,26048,26048,26048,2.604800e+04
unique,NaN,1650,3442,2,14,1,1,17,NaN,NaN,...,NaN,NaN,53,NaN,NaN,NaN,1,1,359,NaN
top,NaN,marina one residences,23 marina way,apartment,apartment,ocr,strata,3,NaN,NaN,...,NaN,NaN,leasehold/99 years,NaN,NaN,NaN,uncategorized,guarded,2021-10-12,NaN
freq,NaN,668,274,13958,13306,26048,26048,7306,NaN,NaN,...,NaN,NaN,15300,NaN,NaN,NaN,26048,26048,1728,NaN
mean,5.495573e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.278091,12.211609,...,1.321145,103.846138,NaN,2010.926398,462.350636,1248.902096,NaN,NaN,NaN,2.994669e+06
std,2.608064e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.147204,7.020134,...,0.038569,0.049865,NaN,11.915875,423.476259,1066.102020,NaN,NaN,NaN,4.324294e+06
min,1.000122e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,...,1.239337,103.696215,NaN,1799.000000,1.000000,226.000000,NaN,NaN,NaN,5.566000e+05
25%,3.229224e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,7.000000,...,1.296472,103.820262,NaN,2010.000000,130.000000,678.000000,NaN,NaN,NaN,1.331000e+06
50%,5.486808e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,11.000000,...,1.313384,103.843081,NaN,2014.000000,366.000000,958.000000,NaN,NaN,NaN,1.851800e+06
75%,7.764926e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,18.000000,...,1.341210,103.879673,NaN,2016.000000,646.000000,1356.000000,NaN,NaN,NaN,2.948000e+06


## 缺失值

## built_year:10043,model 1280, bathrooms 1236,no_of_units:1042,bedrooms 404,tenure 287

## 怎么填补缺失值？

built_year和no_of_units可以用公寓名称等定位出来，有没有有数据的，进行填补
或者众数or 中位数？ 

model是type的细分，type没有缺失，可以用相应的type来填补model?

bedrooms可以用面积推理出来，或者再加上公寓名称什么的

tenure可以看公寓名称？或者公寓类型？

to do: built_year变成leasing date-built year

to do: model和type的关系

## 通过unique发现值只有一个的变量，没有意义：market_segment,type_of_area,eco_category,accessibility可以舍去

In [99]:
data_raw.nunique()

listing_id        26048
name               1650
street             3442
type                  2
model                14
market_segment        1
type_of_area          1
bedrooms             17
bathrooms             9
district             27
region                5
planning_area        38
subszone            193
lat                3118
lng                3118
tenure               53
built_year           60
no_of_units         526
area_size          1178
eco_category          1
accessibility         1
date_listed         359
price              4704
dtype: int64

In [8]:
data_raw['street'].unique()

array(['57 cove way ', '10 moulmein rise ', '2 suffolk walk ', ...,
       '71 oxley rise ', '301 jalan bukit ho swee ',
       '12b cairnhill rise '], dtype=object)

street很多，有意义吗？怎么处理？

In [44]:
data_raw['type'].unique()

array(['condominium', 'apartment'], dtype=object)

type名义变量，可以One-hot

In [45]:
data_raw['model'].unique()

array(['condominium', 'apartment', 'executive condominium', nan,
       'walk-up apt', 'townhouse', 'soho', 'penthouse', 'duplex',
       'high rise', 'residential with commercial on level 1', 'with pool',
       'low rise', 'shophouse', 'strata terrace'], dtype=object)

先看分布,null补全，decision-tree

model是type的细分，怎么处理？

In [46]:
data_raw['market_segment'].unique()

array(['ocr'], dtype=object)

In [47]:
data_val['market_segment'].unique()

array(['ocr'], dtype=object)

market_segment只有ocr，没有意义

In [48]:
data_raw['type_of_area'].unique()

array(['strata'], dtype=object)

In [49]:
data_val['type_of_area'].unique()

array(['strata'], dtype=object)

type_of_area只有strata，没有意义

In [50]:
data_raw['bedrooms'].unique()

array(['3', '4', '2', '1', '3+1', '1+1', '2+1', nan, '5', '4+1', '7',
       '5+1', '6', '3+2', '2+2', '9', '8', '4+2'], dtype=object)

to do:具体看3+1的表示

In [51]:
type(data_raw['bedrooms'][0])

str

'bedrooms' string先进行数据处理2+1是什么意思？然后变成分段变量

In [52]:
data_raw['bathrooms'].unique()

array([ 4.,  3.,  2.,  1., nan,  5.,  6.,  7.,  8., 10.])

In [53]:
type(data_raw['bathrooms'][0])

numpy.float64

float变成Int

In [54]:
data_raw['district'].unique()

array([ 4, 11, 14, 15, 27, 23, 19,  7,  2, 10, 18, 16,  1,  6,  3,  5,  9,
       28, 13, 21, 17,  8, 22, 12, 20, 25, 26])

In [55]:
type(data_raw['district'][0])

numpy.int64

In [56]:
data_raw['region'].unique()

array(['central region', 'east region', 'north region', 'west region',
       'north-east region'], dtype=object)

In [57]:
data_raw['planning_area'].unique()

array(['southern islands', 'novena', 'bedok', 'sembawang', 'bukit batok',
       'sengkang', 'downtown core', 'newton', 'tampines', 'kallang',
       'bukit merah', 'choa chu kang', 'bukit timah', 'outram',
       'queenstown', 'river valley', 'singapore river', 'toa payoh',
       'hougang', 'tanglin', 'pasir ris', 'geylang', 'museum', 'clementi',
       'serangoon', 'jurong east', 'orchard', 'yishun', 'marine parade',
       'bukit panjang', 'jurong west', 'ang mo kio', 'rochor', 'bishan',
       'punggol', 'woodlands', 'mandai', 'changi'], dtype=object)

In [97]:
data_raw['subszone'].unique()[:10]

array(['sentosa', 'moulmein', 'kaki bukit', 'frankel',
       'sembawang springs', 'hillview', 'compassvale', 'city hall',
       'anson', 'orange grove'], dtype=object)

In [59]:
data_raw['lat'].unique()

array([1.23933663, 1.31953349, 1.31565586, ..., 1.28162632, 1.34241264,
       1.30899132])

In [60]:
data_raw['lng'].unique()

array([103.83748719, 103.84702986, 103.84444472, ..., 103.78486945,
       103.77653939, 103.83828467])

In [61]:
type(data_raw['lng'][0])

numpy.float64

上面这几个都是描述地理位置的

In [62]:
data_raw['tenure'].unique()

array(['leasehold/99 years', 'freehold', nan, '999 years',
       '99 years from 18/04/2016', '956 years from 27/05/1928',
       '99 years from 12/08/2002', 'leasehold/60 years',
       '99 years from 14/08/2018', '9999 years from 02/06/1995',
       '99 years from 09/03/2016', 'leasehold/103 years',
       '99 years from 10/07/2013', '99 years from 25/06/2008',
       '99 years from 03/07/2019', '99 years from 12/07/2018',
       '946 years from 01/01/1938', '99 years',
       '99 years from 29/06/2015', '99 years from 16/04/2012',
       '99 years from 19/03/2001', '99 years from 04/12/2012',
       '99 years from 27/03/2018', '110 years from 01/11/2017',
       '99 years from 11/02/2009', '99 years from 10/12/2018',
       '99 years from 24/12/2018', '947 years from 05/10/1934',
       '998 years from 12/10/1885', '99 years from 01/09/1991',
       '99 years from 01/03/1991', '99 years from 21/03/2006',
       '947 years from 15/05/1937', '99 years from 13/03/2017',
       '99 year

In [63]:
type(data_raw['tenure'][0])

str

929 years from 01/01/1953但是建造年限是1997，要进行相应的转化，提取出from的字段减去建造年份的间隔，得到新的tenure.但是大体上可以分成
99年，999年，freehold，可以聚类？

to do:tenure的年份到底是啥关系？

In [64]:
data_raw['built_year'].unique()

array([2011., 1999., 2012.,   nan, 2014., 2020., 2002., 2016., 2008.,
       2018., 2017., 2009., 1978., 1993., 2019., 2015., 2013., 2007.,
       2010., 1989., 1996., 1930., 1985., 2021., 1998., 1995., 2005.,
       2000., 1974., 2004., 2003., 2001., 1997., 2006., 1937., 1994.,
       1986., 1977., 1984., 1970., 1976., 1990., 1992., 1981., 1988.,
       1991., 1966., 1983., 1975., 1979., 1980., 1960., 1972., 1982.,
       1799., 1971., 1967., 1910., 1965., 1987., 1968.])

In [65]:
type(data_raw['built_year'][0])

numpy.float64

float64变成int

In [68]:
type(data_raw['no_of_units'][0])

numpy.float64

In [95]:
data_raw['no_of_units'].unique()[:5]

array([151.,  24., 235., 237.,  36.])

no_of_units - 公寓中的单元总数,描述这个公寓大不大，可以聚类或者分段？

to do:分段？decision tree决定要不要分段，不需手动，看分布：平均？

In [72]:
data_raw['area_size'].unique()

array([2336., 1259., 1959., ..., 2880.,  833., 5743.])

In [73]:
type(data_raw['area_size'][0])

numpy.float64

In [74]:
data_raw['eco_category'].unique()

array(['uncategorized'], dtype=object)

In [75]:
data_val['eco_category'].unique()

array(['uncategorized'], dtype=object)

eco_category,没用,或者自己找数据补充进去

to do:eco_category去掉

In [76]:
data_raw['accessibility'].unique()

array(['guarded'], dtype=object)

In [77]:
data_val['accessibility'].unique()

array(['guarded'], dtype=object)

accessibility 都是guarded，没用

In [96]:
data_raw['date_listed'].unique()[:10]

array(['2021-05-04', '2021-05-07', '2021-06-30', '2022-01-02',
       '2021-12-24', '2022-01-04', '2022-01-05', '2021-12-17',
       '2021-12-25', '2021-09-24'], dtype=object)

在 SRX 上发布的日期，格式变成timestamp?

to do:大部分时间是21年1月份以后，长尾效应。受经济影响不大

In [78]:
data_raw['price'].unique()

array([5390000., 2310000., 5279500., ..., 1230500., 2383100., 4160000.])

In [79]:
type(data_raw['price'][0])

numpy.float64

## 关于其他数据的讨论

15分钟生活圈，可以试一下数据1km-3km，统计里面学校/商圈的个数，可以用球面数据https://www.movable-type.co.uk/scripts/latlong.html

数据的补全

type model关系探索

bedroom 3+1这种怎么处理

地理位置各种变量怎么用？

看一下test数据集跟train数据集的overlap，如果公寓名称等能对应起来，就直接出来价格--short cut